In [341]:
import os
import sys
import re
import pandas as pd
import zipfile
from enum import Enum, EnumMeta
from Tables import Tables
from Spells import Spells
from Items import Items
from SegmentTable import BaseOps, BaseTable, Segments
from IPython.display import display
from random import sample

In [342]:
class DataPiece(BaseOps):
    
    def __init__(self, buffer, position, length, mismatch=0):
        self.buffer = buffer
        self.position = position
        self.length = length
        self.mismatch = mismatch
        self.end = self.position + self.length
        self.signed = False
        self.order = 'little'
    
    def getBytes(self):
        return self.buffer[self.mismatch + self.position:
                           self.mismatch + self.end]
    
    def getValue(self):
        return self.bytes2Num(self.getBytes(),
                              self.signed,
                              self.order)
    
    def getString(self):
        return re.sub(r'[\x00-\x1f\x7f-\x9f]', '', str(self.getBytes(), encoding='utf-8'))
        
    def setValue(self, new_value):
        if isinstance(new_value, Enum):
            new_value = new_value.value
        self.buffer[self.mismatch + self.position: 
                    self.mismatch + self.end] = self.num2Bytes(new_value,
                                                               self.length,
                                                               self.signed,
                                                               self.order)    
        
    def incValue(self, inc_value):
        self.setValue(self.getValue() + inc_value)

In [343]:
class SegmentStructure(BaseOps, Enum):
    
    def __init__(self, header_length, mismatch, 
                 offset_position, offset_length, 
                 count_position, count_length, 
                 length_position, length_length, 
                 record_length, spacing, 
                 abs_count=None, abs_length=None, extra_segment=None):
        self.header_length = header_length
        self.mismatch = mismatch
        self.offset_position = offset_position
        self.offset_length = offset_length
        self.count_position = count_position
        self.count_length = count_length
        self.length_position = length_position
        self.length_length = length_length
        self.record_length = record_length
        self.spacing = spacing
        self.abs_count = abs_count
        self.abs_length = abs_length
        self.extra_segment = extra_segment
    
    def getOffset(self, parent_buffer, parent_mismatch=0):
        if self.offset_position is None:
            return parent_mismatch
        return self.bytes2Num(parent_buffer[self.offset_position + parent_mismatch:
                                            self.offset_position + parent_mismatch + self.offset_length])
    
    def getCount(self, parent_buffer, parent_mismatch=0):
        if self.abs_count is not None:
            return self.abs_count
        elif self.count_position is not None:
            return self.bytes2Num(parent_buffer[self.count_position + parent_mismatch:
                                                self.count_position + parent_mismatch + self.count_length])
        else:
            return None
        
    def getLength(self, parent_buffer, parent_mismatch=0):
        if self.abs_length is not None:
            return self.abs_length
        elif self.length_position is not None:
            return self.bytes2Num(parent_buffer[self.length_position + parent_mismatch:
                                                self.length_position + parent_mismatch + self.length_length])
        _count = self.getCount(parent_buffer, parent_mismatch)
        if _count is not None:
            return _count * self.record_length
        else:
            return None        
    
    def getBufferCopy(self, parent_buffer, parent_mismatch=0):
        return parent_buffer[self.getOffset(parent_buffer, 
                                            parent_mismatch) + parent_mismatch:
                             self.getOffset(parent_buffer, 
                                            parent_mismatch) + parent_mismatch + self.getLength(parent_buffer, 
                                                                                                parent_mismatch)]
    
    def updateOffset(self, parent_buffer, new_offset, parent_mismatch=0):
        if self.getLength(parent_buffer, parent_mismatch) == 0:
            new_offset = 0
        parent_buffer[self.offset_position + parent_mismatch:
                      self.offset_position + parent_mismatch + self.offset_length] = self.num2Bytes(new_offset, 
                                                                                                    self.offset_length)
    
    def updateCount(self, parent_buffer, new_count, parent_mismatch=0):
        if self.count_position is not None:
            parent_buffer[self.count_position + parent_mismatch:
                          self.count_position + parent_mismatch + self.count_length] = self.num2Bytes(new_count, 
                                                                                                      self.count_length)
            
    def incCount(self, parent_buffer, inc_count=1, parent_mismatch=0):
        self.updateCount(parent_buffer, self.getCount(parent_buffer, parent_mismatc) + inc_count, parent_mismatc)
    
    def updateBuffer(self, parent_buffer, new_buffer, parent_mismatch=0):
        parent_buffer[self.getOffset(parent_buffer, 
                                     parent_mismatch) + parent_mismatch:
                      self.getOffset(parent_buffer, 
                                     parent_mismatch) + parent_mismatch + self.getLength(parent_buffer, 
                                                                                         parent_mismatch)] = new_buffer
    
    def getOffsetRef(self, parent_buffer, parent_mismatch=0):
        if self.offset_position is not None:
            return DataPiece(parent_buffer, self.offset_position, self.offset_length, parent_mismatch)
        else:
            return None
    
    def getCountRef(self, parent_buffer, parent_mismatch=0):
        if self.count_position is not None:
            return DataPiece(parent_buffer, self.count_position, self.count_length, parent_mismatch)
        else:
            return None
    
    def getLengthRef(self, parent_buffer, parent_mismatch=0):
        if self.length_position is not None:
            return DataPiece(parent_buffer, self.length_position, self.length_length, parent_mismatch)
        else:
            return None

In [403]:
class BaseSegment(BaseOps):
    
    class Subs(SegmentStructure):
        pass
    
    class Record(BaseTable):
        pass
    
    def __init__(self, filename, name, parent, buffer, header_length, mismatch, 
                 offset_ref, count_ref, length_ref, record_length, spacing,
                 abs_count=None, abs_length=None, extra_segment=None):
        self.filename = filename
        self.name = name
        self.parent = parent
        self.buffer = buffer
        self.header_length = header_length
        self.mismatch = mismatch
        self.offset_ref = offset_ref
        self.count_ref = count_ref
        self.length_ref = length_ref
        self.record_length = record_length
        self.spacing = spacing
        self.abs_count = abs_count
        self.abs_length = abs_length
        self.extra_segment = extra_segment
        self.header = None
        self.tail = None
        self.previous = None
        self.next = None
        self.children = list()
        self._read()
        self.split()

        
    @property
    def count(self):
        if self.abs_count:
            return self.abs_count
        elif self.count_ref is not None:
            return self.count_ref.getValue()
        else:
            return None
    
    @property
    def offset(self):
        if self.offset_ref:
            return self.offset_ref.getValue()
        else:
            return 0
        
    @property
    def readLength(self):
        if self.abs_length is not None:
            return self.abs_length
        elif self.length_ref is not None:
            return self.length_ref.getValue()
        elif self.count is not None:
            return self.count * self.record_length
        else:
            return None
        
        
    @property
    def calLength(self):
        _len = len(b''.join(self.records))
        if self.header is not None:
            _len += len(self.buffer[self.header])
        if self.tail is not None:
            _len += len(self.buffer[self.tail])
        if self.extra_segment is not None:
            _len += len(self.buffer[self.extra_segment])
        for _seg in self.children:
            _len += _seg.calLength
        return _len
    
    @property
    def combined_records(self):
        return b''.join(self.records)
     
        
    def _read(self):
        self.records = list()
        for _i in range(self.count):
            _start = self.header_length + self.record_length * _i
            _end = _start + self.record_length
            _record = self.buffer[_start: _end]
            self.records.append(_record)
    
    @property
    def data(self):
        _cols = self.Record.__members__.keys()
        self._data = pd.DataFrame(columns=_cols)
        for _record in self.records:
            _dict = dict(zip(_cols, [_.getRepr(_record) for _ in self.Record]))
            self._data = self._data.append(_dict, ignore_index=True) 
        return self._data
        
    
    def split(self):
        self.children = list()
        _end = 0
        for _index, _item in enumerate(self.Subs):
            _name = _item.name
            _offset = _item.getOffset(self.buffer, self.header_length)
            _length = _item.getLength(self.buffer, self.header_length)
            setattr(self, _name, 
                    getattr(Appendix, _name)(self.filename, _name, self, 
                                             _item.getBufferCopy(self.buffer, self.header_length),
                                             _item.header_length, 
                                             _item.mismatch,
                                             _item.getOffsetRef(self.buffer, self.header_length),
                                             _item.getCountRef(self.buffer, self.header_length),
                                             _item.getLengthRef(self.buffer, self.header_length),
                                             _item.record_length,
                                             _item.spacing,
                                             _item.abs_count,
                                             _item.abs_length,
                                             _item.extra_segment
                                            )
                   )
            self.children.append(getattr(self, _name))    
            if _end < _offset + _length:
                _end = _offset + _length
        if _end == 0:
            _end = self.readLength
        self.header = slice(None, self.header_length)
        self.tail = slice(_end, None)
        
    '''    
    def _pack(self):
        _sub_buffer = bytearray()
        _extra_spacing = len(self.combined_records) + len(self.buffer[self.header])
        if self.extra_segment is not None:
            _extra_spacing += len(self.buffer[self.extra_segment])
        for _sub in self.children:
            if _sub.offset_ref is not None:
                if _sub.count == 0:
                    _sub.offset_ref.setValue(0)
                else:
                    _sub.offset_ref.setValue(len(_sub_buffer) + _extra_spacing)
            if _sub.count_ref is not None:
                _sub.count_ref.setValue(_sub.count)
            if _sub.length_ref is not None:
                _sub.length_ref.setValue(_sub.calLength())
            _sub_buffer.extend(_sub.pack())
        _buffer = bytearray()
        _buffer.extend(self.buffer[self.header])
        _buffer.extend(self.combined_records)
        if self.extra_segment is not None:
            _buffer.extend(self.buffer[self.extra_segment])
        _buffer.extend(_sub_buffer)
        _buffer.extend(self.buffer[self.tail])
        return _buffer
    '''

    def pack(self, *spacing):
        if not spacing:
            spacing = self.spacing
        else:
            spacing = spacing[0]
        _sub_buffer = bytearray()
        _spacing = len(self.combined_records) + len(self.buffer[self.header])
        if self.extra_segment is not None:
            _spacing += len(self.buffer[self.extra_segment])
        for _sub in self.children:
            _sub_buffer.extend(_sub.pack(len(_sub_buffer) + _spacing))
        if self.offset_ref is not None:
            if self.count == 0:
                pass
            else:
                self.offset_ref.setValue(spacing)
        if self.count_ref is not None:
            self.count_ref.setValue(self.count)
        if self.length_ref is not None:
            self.length_ref.setValue(self.calLength)
        _buffer = bytearray()
        _buffer.extend(self.buffer[self.header])
        _buffer.extend(self.combined_records)
        if self.extra_segment is not None:
            _buffer.extend(self.buffer[self.extra_segment])
        _buffer.extend(_sub_buffer)
        _buffer.extend(self.buffer[self.tail])
        return _buffer
    
    @staticmethod
    def searchName(_value, table):
        if not isinstance(_value, str):
            _value = str(_value, encoding='UTF-8')
        _value = _value.replace('\x00', '')
        for _cls in table.__dict__.values():
            if isinstance(_cls, EnumMeta):
                for _item in _cls:
                    if _item.value == _value:
                        return _item.name, _cls.__name__
        else:
            return _value, None

In [539]:
class Appendix:
    
    class KnownSpell(BaseSegment):
        
        class Record(BaseTable):

            SPELL = (0x0000, 8)
            LEVEL = (0x0008, 2)
            TYPE = (0x000a, 2)
            
        SPELL_TEMPLATE = bytearray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        SPELL_PATTERN = re.compile(r'(?P<type>\w{4})(?P<level>\d)(?P<id>\d+)')
        SPELL_TYPE = {'SPWI': 1,
                      'SPCL': 0,
                      'SPIN': 2,
                      'SPPR': 3,
                      1: 'Wizard',
                      0: 'Cleric',
                      2:' Innate',
                      3: 'Special'}
            
        def prepareTemplate(self, spell):
            _string = re.match(self.SPELL_PATTERN, spell.value)
            _template = self.SPELL_TEMPLATE[:]
            _spell = self.bytes2Num(bytearray(spell.value, encoding='UTF-8'))
            _level = int(_string.group('level')) - 1
            if _level < 0:
                raise('Wait to figure out Lv.10 spells!')
            _type = self.SPELL_TYPE[_string.group('type')]
            self.Record.SPELL.setValue(_template, _spell)
            self.Record.LEVEL.setValue(_template, _level)
            self.Record.TYPE.setValue(_template, _type)
            return _template
        
        def _insert(self, spell):
            _count = 0
            _template = self.prepareTemplate(spell)
            if not _template in self.records:
                self.records.append(_template)
                _count += 1
            return _count
        
        def _delete(self, spell):
            _count = 0
            _template = self.prepareTemplate(spell)
            if _template in self.records:
                self.records.pop(self.records.index(_template))
                _count += 1
            return _count
        
        def insert(self, spell):
            _count = 0
            if isinstance(spell, EnumMeta):
                for _spell in spell:
                    _count += self._insert(_spell)
            else:
                _count = self._insert(spell)
            if self.count_ref:
                self.count_ref.incValue(_count)
            self.sort()
                
        def delete(self, spell):
            _count = 0
            if isinstance(spell, EnumMeta):
                for _spell in spell:
                    _count += self._delete(_spell)
            else:
                _count = self._delete(spell)
            if self.count_ref:
                self.count_ref.incValue(-_count)
            self.sort()
                
        def sort(self):
            self.records = sorted(self.records, key=lambda x: (self.Record.TYPE.getValue(x, 0),
                                                               self.Record.LEVEL.getValue(x, 0)))
            
        @property
        def data(self):
            self._data = super().data
            _name = self.Record.SPELL.name
            _length = self.Record.SPELL.length
            self._data[_name] = self._data[_name].apply(lambda x: self.searchName(self.num2Bytes(int(x), _length),
                                                                                 table=Spells)[0])
            return self._data
                    
    class MemSpell(BaseSegment):
        pass
    
    class MemorizedSpell(BaseSegment):
        pass
    
    class ItemSlot(BaseSegment):
        
        class Position(BaseTable):
        
            HELMET = (0x0000, 2)
            ARMOR = (0x0002, 2)
            SHIELD = (0x0004, 2)
            GLOVES = (0x0006, 2)
            RINGS = (0x0008, 4)
            AMULET = (0x000c, 2)
            BELT = (0x000e, 2)
            BOOTS = (0x0010, 2)
            WEAPONS = (0x0012, 4)
            EXTRAWEAPONS = (0x0016, 4)
            QUIVERS = (0x001a, 6)
            EXTRAQUIVER = (0x0020, 2)
            CLOAK = (0x0022, 2)
            QUICK = (0x0024, 6)
            INVENTORY = (0x002a, 32) 
            MAGICWEAPON = (0x004a, 2)
            SELECTEDWEAPON = (0x004c, 2)
            SELECTEDABILITY = (0x004e, 2)
            
        class Category(BaseOps):
            
            def __init__(self, capacity):
                self.index = list()
                self.item = list()
                self.code = list()
                self.capacity = capacity
                
            def add(self, _index, _item):
                if len(self.index) < self.capacity:
                    _code = self.num2Bytes(_index, 2)
                    self.code.append(_code)
                    self.item.append(_item)
                    self.index.append(_index)
                    return True
                else:
                    return False
                    
            def fillna(self):
                self.code.extend([b'\xff\xff'] * (self.capacity - len(self.index)))
                return self.code
                                  
            
            
        def delete(self, index):
            self.records[index] = b'\xff\xff'
            
        def place(self, inventory, items):
            self.records = list()
            
            self.helmet = self.Category(1)
            self.armor = self.Category(1)
            self.shield = self.Category(1)
            self.gloves = self.Category(1)
            self.rings = self.Category(2)
            self.amulet = self.Category(1)
            self.belt = self.Category(1)
            self.boots = self.Category(1)
            self.weapons = self.Category(2)
            self.extraweapons = self.Category(2)
            self.quivers = self.Category(3)
            self.extraquiver = self.Category(1)
            self.cloak = self.Category(1)
            self.quick = self.Category(3)
            self.inventory = self.Category(16)
            self.magicweapon = self.Category(1)
            self.selectedweapon = self.Category(1)
            self.selectedability = self.Category(1)

            occupied = { 'CLOAKS': self.cloak,
                         'ARROWS': self.quivers,
                         'TWO_HANDED_SWORDS': self.weapons,
                         'DROW_ITEMS': self.inventory,
                         'QUIVERS': self.quivers,
                         'POTIONS': self.quick,
                         'Exceptions': self.inventory,
                         'DAGGERS': self.weapons,
                         'WANDS': self.quick,
                         'RINGS': self.rings,
                         'CROSSBOWS': self.weapons,
                         'PLOT_ITEMS': self.inventory,
                         'HAMMERS': self.weapons,
                         'SHIELDS': self.shield,
                         'SECRET_PANTALOON_STUFF': self.inventory,
                         'STAVES': self.weapons,
                         'SPEARS': self.weapons,
                         'ROBES': self.armor,
                         'CHAINMAILS': self.armor,
                         'BOWS': self.weapons,
                         'HALBERDS': self.weapons,
                         'BOLTS': self.quivers,
                         'HELMS': self.helmet,
                         'DRAGON_SCALES': self.inventory,
                         'IWD': self.inventory,
                         'AXES': self.weapons,
                         'BOOKS': self.inventory,
                         'SLINGS': self.weapons,
                         'KEYS': self.inventory,
                         'BELTS': self.belt,
                         'BOOTS': self.boots,
                         'TOMES': self.inventory,
                         'SCROLLS': self.inventory,
                         'BRACERS': self.gloves,
                         'BAGS': self.inventory,
                         'ONE_HANDED_SWORDS': self.weapons,
                         'AMULETS': self.amulet,
                         'BLUNT_WEAPONS': self.weapons,
                         'BULLETS': self.quivers,
                         'DARTS': self.quivers,
                         'RODS': self.quick,
                         'FULL_PLATE_ARMORS': self.armor,
                         'FAMILIARS': self.inventory,
                         'MISC': self.inventory,
                         'COMPONENTS': self.inventory,
                         'LEATHER_ARMORS': self.armor          
                        }
            for _index, (_category, _item) in enumerate(zip(inventory, items)):
                if _category in occupied.keys():
                    if not occupied[_category].add(_index, _item):
                        for _other in [self.inventory, self.quick, self.quivers, self.shield, self.helmet, 
                                       self.armor, self.gloves, self.rings, self.amulet, self.belt, self.boots,
                                       self.weapons, self.cloak, self.extraweapons]:
                            if _other.add(_index, _item):
                                break
                else:
                    self.inventory.add(_index, _item) 
                    
            for _item in self.Position:
                _category = getattr(self, _item.name.lower())
                self.records.extend(_category.fillna())
            
    class Item(BaseSegment):
        
        GOODITEMS =  [Items.AMULETS.Amulet_of_Power, 
                      Items.AMULETS.Amulet_of_the_Master_Harper,
                      Items.AXES.Axe_of_the_Unyielding_PLUS5,
                      Items.BELTS.Belt_of_Inertial_Barrier,
                      Items.BLUNT_WEAPONS.Flail_of_Ages_PLUS5,
                      Items.BOOTS.Boots_of_Speed,
                      Items.BOWS.Short_Bow_of_Gesen,
                      Items.BOWS.Taralash_PLUS5,
                      Items.BRACERS.Gauntlets_of_Extraordinary_Specialization,
                      Items.CHAINMAILS.Aslyferund_Elven_Chain_PLUS5,
                      Items.CLOAKS.Cloak_of_Mirroring,
                      Items.CLOAKS.Cloak_of_Bravery,
                      Items.ROBES.Robe_of_the_Good_Archmagi,
                      Items.ROBES.Robe_of_the_Neutral_Archmagi,
                      Items.CROSSBOWS.Firetooth_PLUS5,
                      Items.DAGGERS.Dagger_of_the_Star_PLUS5,
                      Items.HALBERDS.Ravager_PLUS6,
                      Items.HAMMERS.Runehammer_PLUS5,
                      Items.HELMS.Circlet_of_Netheril_1,
                      Items.HELMS.Kiels_Helmet,
                      Items.HELMS.Helm_of_Charm_Protection,
                      Items.FULL_PLATE_ARMORS.Enkidus_Full_Plate_PLUS3,
                      Items.RINGS.Ring_of_Wizardry,
                      Items.RINGS.Ring_of_Holiness,
                      Items.RINGS.Ring_of_Gaxx,
                      Items.RINGS.Ring_of_Acuity,
                      Items.RINGS.Heartwood_Ring,
                      Items.RINGS.Ring_of_AntiVenom,
                      Items.SHIELDS.Reflection_Shield_PLUS1,
                      Items.SHIELDS.Shield_of_Harmony_PLUS2,
                      Items.SLINGS.Erinne_Sling_PLUS5,
                      Items.SPEARS.Spear_of_the_Unicorn_PLUS2,
                      Items.STAVES.Staff_of_the_Magi,
                      Items.STAVES.Staff_of_the_Ram_PLUS6,
                      Items.ONE_HANDED_SWORDS.Blackrazor_Long_Sword_PLUS3,
                      Items.ONE_HANDED_SWORDS.Scimitar_of_Speed_PLUS2_Belm,
                      Items.ONE_HANDED_SWORDS.Adjatha_the_Drinker_Long_Sword_PLUS2,
                      Items.ONE_HANDED_SWORDS.Celestial_Fury_Katana_PLUS3,
                      Items.ONE_HANDED_SWORDS.Short_Sword_of_Mask_PLUS5,
                      Items.ONE_HANDED_SWORDS.Angurvadal_PLUS5,
                      Items.ONE_HANDED_SWORDS.The_Answerer_PLUS4,
                      Items.ONE_HANDED_SWORDS.Scimitar_PLUS5_Defender,
                      Items.TWO_HANDED_SWORDS.Psions_Blade_PLUS5,
                      Items.QUIVERS.Bag_of_Plenty_PLUS2,
                      Items.QUIVERS.Case_of_Plenty_PLUS2,
                      Items.QUIVERS.Quiver_of_Plenty_PLUS2]

        GOODITEMS = [Items.RINGS.Ring_of_Wizardry,
                     Items.RINGS.Ring_of_Acuity,
                     Items.RINGS.Ring_of_Holiness,
                     Items.HELMS.Helm_of_Balduran,
                     Items.HELMS.Helm_of_Charm_Protection,
                     Items.HELMS.Kiels_Helmet,
                     Items.HELMS.Pale_Green_Ioun_Stone,
                     Items.BRACERS.Gauntlets_of_Weapon_Expertise,
                     Items.BOOTS.Boots_of_Speed,
                     Items.AMULETS.Amulet_of_Power,
                     Items.SHIELDS.Large_Shield_PLUS1_PLUS4_vs_Missiles,
                     #Items.SHIELDS.Kiels_Buckler,
                     Items.FULL_PLATE_ARMORS.The_Magma_Bulwark_PLUS2,
                     Items.CHAINMAILS.Elven_Chain_Mail,
                     Items.ROBES.Robe_of_the_Good_Archmagi,
                     Items.ROBES.Robe_of_the_Neutral_Archmagi,
                     Items.SLINGS.Sling_of_UNERRING_ACCURACY,
                     Items.CROSSBOWS.Heavy_Crossbow_of_Accuracy,
                     Items.CROSSBOWS.Light_Crossbow_of_Speed,
                     Items.BOWS.Long_Bow_of_Marksmanship,
                     Items.ONE_HANDED_SWORDS.Scimitar_PLUS2,
                     Items.STAVES.Staff_Spear_PLUS2,
                     Items.BLUNT_WEAPONS.The_Stupefier_PLUS1,
                     Items.ONE_HANDED_SWORDS.Long_Sword_PLUS1_Flame_Tongue,
                     #Items.DAGGERS.Boomerang_Dagger_PLUS2,
                     Items.ONE_HANDED_SWORDS.Bastard_Sword_PLUS1_Albruin,
                     Items.AXES.Klogarath_PLUS4,
                     Items.QUIVERS.Bag_of_Plenty_PLUS1,
                     Items.QUIVERS.Case_of_Plenty_PLUS1,
                     Items.QUIVERS.Quiver_of_Plenty_PLUS1,
                     Items.BAGS.Bag_of_Holding,
                     Items.BELTS.Belt_of_Inertial_Barrier,
                     Items.CLOAKS.Cloak_of_Bravery,
                     Items.CLOAKS.Cloak_of_NonDetection]
    
        class Record(BaseTable):
            NAME = (0x0000, 8)
            EXPIRATION = (0x0008, 1)
            ELAPSED = (0x0009, 1)
            QUANTITY_1 = (0x000a, 2)
            QUANTITY_2 = (0x000c, 2)
            QUANTITY_3 = (0x000e, 2)
            IDENTIFIED = (0x000f, 1)
            UNSTEALABLE = (0x0010, 1)
            STOLEN = (0x0011, 1)
            UNDROPPABLE = (0x0012, 1)

        MAX_COUNT = 40

        ITEM_TEMPLATE = bytearray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
        
        @property
        def data(self):
            _col = list(self.Record.__members__.keys()) + ['CATEGORY']
            self._data = pd.DataFrame(columns=_col)
            for _record in self.records:
                _dict = dict()
                for _item in self.Record:
                    if _item == self.Record.NAME:
                        _dict[self.Record.NAME.name], _dict['CATEGORY'] = self.searchName(_item.getBytes(_record),
                                                                                          table=Items)
                    else:
                        _dict[_item.name] = _item.getValue(_record)
                self._data = self._data.append(_dict, ignore_index=True)
            for col in self._data.columns:
                if not col in [self.Record.NAME.name, 'CATEGORY']:
                    self._data[col] = self._data[col].astype(int)
            return self._data
        
        def _insert(self, item, quantity=1, unique=False):
            _template = self.ITEM_TEMPLATE[:]
            self.Record.NAME.setValue(_template, self.bytes2Num(bytearray(item.value, encoding='UTF-8')))
            self.Record.QUANTITY_1.setValue(_template, quantity) 
            if self.count < self.MAX_COUNT:
                if unique == False or not _template in self.records:
                    self.records.insert(0, _template)
                    if self.count_ref is not None:
                        self.count_ref.incValue(1)
                    return True
                else:
                    return False
            else:
                if unique == False or not _template in self.records:
                    self.records = self.records[:-1]
                    self.records.insert(0, _template)
                    return True
                else:
                    return False
                    
        def insert(self, item, quantity=1, unique=False):
            if self._insert(item, quantity, unique) == True:
                self.place()
        
        def delete(self, item, unique=False):
            _code = self.bytes2Num(item.value.encode(), self.Record.NAME.length)
            _items = [self.Record.NAME.getValue(_record) for _record in self.records]
            _index = [_i for _i, _item in enumerate(_items) if _item == _code]
            if _index:
                if unique == True:
                    _index = _index[0:1]
                for _i in _index:
                    self.records.pop(_i)
                if self.count_ref is not None:
                    self.count_ref.incValue(-len(_index))    
                    
        def place(self):
            categories = self.data['CATEGORY']
            items = self.data['NAME']
            self.parent.ItemSlot.place(categories, items)
            
        def optimize(self, size=None):
            if size is None:
                size = self.MAX_COUNT
            if size > len(self.GOODITEMS):
                size = len(self.GOODITEMS)
            for _item in sample(self.GOODITEMS, size):
                self._insert(_item) 
            self.place()

    class Effect(BaseSegment):
        
        class WeaponOptima(Enum):
            BastardSword = 5
            LongSword = 5
            ShortSword = 5
            Axe = 5
            TwoHandedSword = 5
            Katana = 5
            ScimitarWakizashiNinjaTo = 5
            Dagger = 5
            WarHammer = 5
            Spear = 5
            Halberd = 5
            FlailMorningstar = 5
            Mace = 5
            QuarterStaff = 5
            Crossbow = 5
            LongBow = 5
            ShortBow = 5
            Darts = 5
            Sling = 5
            #Blackjack = 5
            #Gun = 5
            #MartialArts = 5
            TwoHandedWeaponSkill = 2
            SwordandShieldSkill = 2
            SingleWeaponSkill = 2
            TwoWeaponSkill = 3
            Club = 5
            #ExtraProficiency2 = 5
            #ExtraProficiency3 = 5
            #ExtraProficiency4 = 5
            #ExtraProficiency5 = 5
            #ExtraProficiency6 = 5
            #ExtraProficiency7 = 5
            #ExtraProficiency8 = 5
            #ExtraProficiency9 = 5
            #ExtraProficiency10 = 5
            #ExtraProficiency11 = 5
            #ExtraProficiency12 = 5
            #ExtraProficiency13 = 5
            #ExtraProficiency14 = 5
            #ExtraProficiency15 = 5
            #ExtraProficiency16 = 5
            #ExtraProficiency17 = 5
            #ExtraProficiency18 = 5
            #ExtraProficiency19 = 5
            #ExtraProficiency20 = 5
    
        WEAPON_OPCODE = 0xe9
        WEAPON_TEMPLATE = bytearray([0, 0, 0, 0, 0, 0, 0, 0, 233, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 95, 
                                     0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255, 255, 255, 255, 255, 
                                     255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

        class Record(BaseTable):
            OPCODE = (0x0008, 4)
            TARGET = (0x000c, 4, False, 'little', 'Target')
            POWER = (0x0010, 4)
            PARAM_1 = (0x0014, 4, False, 'little', None, 5)
            PARAM_2 = (0x0018, 4, False, 'little', 'WeaponType')
            TIME = (0x001c, 4, False, 'little', 'Time')

        
        def insert(self, weapon, *proficiency):
            if not proficiency:
                proficiency = getattr(self.WeaponOptima, weapon.name).value
            else:
                proficiency = proficiency[0]
            _weapons = [self.Record.PARAM_2.getValue(_record) for _record in self.records if self.Record.OPCODE.getValue(_record) == self.WEAPON_OPCODE]
            _index = [_i for _i, _weapon in enumerate(_weapons) if _weapon == weapon.value]
            if _index:
                self.Record.PARAM_1.setValue(self.records[_index[0]], proficiency)
            else:
                _template = self.WEAPON_TEMPLATE[:]
                self.Record.PARAM_1.setValue(_template, proficiency)
                self.Record.PARAM_2.setValue(_template, weapon.value)
                self.records.insert(0, _template)
                if self.count_ref is not None:
                    self.count_ref.incValue(1)
                    
        def optimize(self, more_weapon=True):
            if more_weapon:
                for _weapon in Tables.WeaponType:
                    self.insert(_weapon)
            else:
                _weapons = [_record for _record in self.records if self.Record.OPCODE.getValue(_record) == self.OPCODE]
                for _weapon in _weapons:
                    _code = self.Record.PARAM_2.getValue(_weapon)
                    _name = Tables.WeaponType(_code).name
                    _proficiency = getattr(self.WeaponOptima, _name).value
                    self.Record.PAMAM_1.setValue(_weapon, _proficiency) 
                    
        @property
        def data(self):
            return super().data[super().data['OPCODE'] == str(self.WEAPON_OPCODE)]

In [540]:
class CRE(BaseSegment):
    
    Record = Segments
    
    class Subs(SegmentStructure, Enum):
        KnownSpell = (0, 0, 0x02a0, 4, 0x02a4, 4, None, None, 12, 52)
        MemSpell = (0, 0, 0x02a8, 4, 0x02ac, 4, None, None, 16, 44)
        MemorizedSpell = (0, 0, 0x02b0, 4, 0x02b4, 4, None, None, 12, 36)
        ItemSlot = (0, 0, 0x02b8, 4, None, None, None, None, 2, 28, 40, 80)
        Item = (0, 0, 0x02bc, 4, 0x02c0, 4, None, None, 20, 24)
        Effect = (0, 0, 0x02c4, 4, 0x02c8, 4,None, None,  264, 16)
            
    @property
    def data(self):
        self._data = super().data.T
        self._data.columns = [self.name]
        return self._data[~self._data.index.str.contains('Ref|Count|Offset')]
    
    @property
    def weapons(self):
        return self.Effect.data
    
    @property
    def items(self):
        return self.Item.data
    
    @property
    def spells(self):
        return self.KnownSpell.data
    
    def optimize(self):
        for _item in Segments:
            _item.optimize(self.records[0]) 
            
    def setValue(self, item, new_value):
        if not isinstance(new_value, int):
            new_value = new_value.value       
        getattr(Segments, item.name).setValue(self.records[0], new_value) 
        
    def optimizeWeapon(self):
        self.Effect.optimize()
        
    def addSpell(self, spell):
        self.KnownSpell.insert(spell)
        
    def removeSpell(self, spell):
        self.KnownSpell.delete(spell)
        
    def addItem(self, item):
        self.Item.insert(item)
        
    def removeItem(self, item):
        self.Item.delete(item)
        
    def optimizeItem(self):
        self.Item.optimize()
    

In [541]:
class CHR:
    
    class Blocks(BaseTable):
        NAME = (0x0008, 32)
        CRE_OFFSET = (0x0028, 4)
        CRE_LENGTH = (0x002c, 4)
         
    def __init__(self, filename):
        self.filename = filename
        self.pwd = os.path.dirname(self.filename)
        self.file = os.path.basename(self.filename)
        self.zipfile = zipfile.ZipFile(self.filename)
        self.file_list = self.zipfile.namelist()
        self.buffer = bytearray(self.zipfile.read(self.file_list[1]))
        self.header_length = 0
        self.mismatch = 0
        self.record_length = 672
        self.spacing = 0
        self.abs_count = 1
        self.abs_length = None
        self.extra_segment = slice(0x02a0, 0x02d4)
        self.read()
        
    @property
    def name(self):
        _name = self.buffer[self.Blocks.NAME.position:
                            self.Blocks.NAME.position + self.Blocks.NAME.length]
        while _name[-1] == 0:
            _name = _name[:-1]
        return _name.decode('UTF-8')
    
    def read(self):
        _offset_ref = DataPiece(self.buffer, self.Blocks.CRE_OFFSET.position, self.Blocks.CRE_OFFSET.length)
        _length_ref = DataPiece(self.buffer, self.Blocks.CRE_LENGTH.position, self.Blocks.CRE_LENGTH.length)
        _start = _offset_ref.getValue()
        _length = _length_ref.getValue()
        _end = _start + _length
        self.header = slice(None, _start)
        self.tail = slice(_end, None)
        self.cre = CRE(self.filename, self.name, None, self.buffer[_start:_end], 
                       self.header_length, self.mismatch, _offset_ref, 
                       None, _length_ref, self.record_length, _start, self.abs_count,
                       self.abs_length, self.extra_segment)
    
    @property
    def weapons(self):
        return self.cre.weapons
    
    @property
    def items(self):
        return self.cre.items
    
    @property
    def spells(self):
        return self.cre.spells
    
    @property
    def data(self):
        return self.cre.data
    
    def optimize(self):
        self.cre.optimize()   
            
    def setValue(self, item, new_value):
        self.cre.setValue(item, new_value)
        
    def optimizeWeapon(self):
        self.cre.optimizeWeapon()
        
    def addSpell(self, spell):
        self.cre.addSpell(spell)
        
    def removeSpell(self, spell):
        self.cre.removeSpell(spell)
        
    def addItem(self, item):
        self.cre.addItem(item)
        
    def removeItem(self, item):
        self.cre.removeItem(item)
        
    def optimizeItem(self):
        self.cre.optimizeItem()
        
    def pack(self, export=False):
        _cre_buffer = self.cre.pack()
        _buffer = self.buffer[self.header]
        _buffer.extend(_cre_buffer)
        _buffer.extend(self.buffer[self.tail])
        bio_buffer = self.zipfile.read(self.file_list[0])
        self.zipfile.close()
        with zipfile.ZipFile(self.filename, 'w') as target:
            for _f, _name in zip([bio_buffer, _buffer], self.file_list):
                target.writestr(_name, _f)
        if export == True:
            with open(self.file_list[1], 'wb') as target:
                target.write(_buffer)

In [515]:
filename = ['01FIGHT.bg1character','02THIEF.bg1character','03MAGE.bg1character','04PALAD.bg1character']
a = [CHR(_) for _ in filename]

In [481]:
a[0].setValue(Segments.Kit, Tables.Kit.GODHELM)
a[2].setValue(Segments.Class, Tables.Class.FIGHTER_MAGE_THIEF)
a[2].setValue(Segments.Kit, Tables.Kit.SHADOWDANCER)
a[3].setValue(Segments.Kit, Tables.Kit.SHADOWDANCER)

In [482]:
a[0].addSpell(Spells.Magic_CLERIC)
a[1].addSpell(Spells.Magic_CLERIC)
a[2].addSpell(Spells.Magic_WIZARD)
a[3].addSpell(Spells.Magic_WIZARD)

In [487]:
[_.optimize() for _ in a]
[_.optimizeItem() for _ in a]
[_.optimizeWeapon() for _ in a]

[None, None, None, None]

In [484]:
[_.pack() for _ in a]

[None, None, None, None]

In [542]:
filename = ['05CLERIC.bg1character','06RANGER.bg1character','07BARB.bg1character','08BARD.bg1character']
a = [CHR(_) for _ in filename]

In [543]:
[_.optimizeItem() for _ in a]
[_.pack() for _ in a]

[None, None, None, None]

In [544]:
len(a[0].cre.ItemSlot.records)

40